# Player types
understanding types of players requires 3 differnt tables: player, extra (stores carousel) and bid (playlist - type of playlist)
### 1) load players with carousel extra (mongo)
### 2) load extras with extra type (mongo)

In [1]:
import pandas as pd

## 1) players

mongo query for players with extra + additional info

fields: **pid,parent_cid,cid,extraIds,templateId,staticVideos,bidId,playlistId**

```
var extrasWithCarousel = db.extra.find({deleted:false, subType:"CAROUSEL"});
var temp, extraIds = [], extraValueOfIds = [];
while ( extrasWithCarousel.hasNext() ){
   temp = extrasWithCarousel.next()._id;
   extraIds.push(temp);
   extraValueOfIds.push(temp.valueOf());
}

var playersWithCarousel = db.player.find({
  deleted:false,
  $or:[
    {extraConfigs:{$exists:true, $elemMatch: {"extraId" : { "$in": extraIds}}}},
    { "templateConfiguration.extraConfigs": {$exists:true, $elemMatch: {"extraId" : { "$in": extraIds}}}}
  ] },
  {_id:1, parentCompanyId:1, companyId:1, templateId:1, templateConfiguration:1, extraConfigs:1, bidId:1,playlistId:1,videoIds:1});
var parentCompanyId,eConfig,eId,templateId,staticVideos,bidId,playlistId,playerIds = [];
print('pid,parent_cid,cid,extraIds,templateId,staticVideos,bidId,playlistId'); //header
while ( playersWithCarousel.hasNext() ){
   temp = playersWithCarousel.next();

   eId = "";
   eConfig = (temp.templateConfiguration && temp.templateConfiguration.extraConfigs && temp.templateConfiguration.extraConfigs != 'NULL_OVERRIDE') ? temp.templateConfiguration.extraConfigs : temp.extraConfigs;
   if (eConfig && typeof eConfig == 'object' && eConfig[0] && eConfig[0].extraId  && eConfig.length > 0) {
      for (i in eConfig) {
         extra = eConfig[i];
	 if (extra.extraId && extraValueOfIds.indexOf(extra.extraId.valueOf()) != -1) {
           eId = extra.extraId.valueOf();
           continue;
         }
      }
   }
   parentCompanyId = temp.parentCompanyId ? temp.parentCompanyId.valueOf() : ''
   templateId = temp.templateId ? temp.templateId.valueOf() : ''
   staticVideos = temp.videoIds ? temp.videoIds.length : 0; //number of static videos
   bidId = temp.bidId ? temp.bidId.valueOf() : '';
   playlistId = temp.playlistId ? temp.playlistId.valueOf() : '';
   print(temp._id.valueOf() + ',' + parentCompanyId + ',' + temp.companyId.valueOf() + ',' +eId + ',' + templateId + ',' + staticVideos + ',' + bidId + ',' + playlistId);
   playerIds.push(temp.valueOf());
}
```

In [34]:
players_df = pd.read_csv('./players_with_carousel_2017_07_30.csv')
print 'number of players with carousel: ', players_df.shape[0]
players_df.head(2)

number of players with carousel:  9485


,pid,parent_cid,cid,extraIds,templateId,staticVideos,bidId,playlistId
0,57223186e4b0af35cc801d63,50d595ec0364e95588c77bd2,50d595ec0364e95588c77bd2,55388b54e4b099bcc1225caa,5668ad07e4b0f82be105f939,0,572192e8e4b0e7aba837de62,NaN
1,5723a735e4b0e6cbf9c142e0,5314327fe4b015a18eb92bab,5314327fe4b015a18eb92bab,5539dff8e4b09639b4f30375,559ba4619fa0c117ee299ea0,0,57111cd1e4b007de6512a839,NaN


## 2) Extras 
**extraId,subType,uiPosition** , currently subType==CAROUSEL always
```
var extrasWithCarousel = db.extra.find({deleted:false, subType:"CAROUSEL"});
var temp;
print('extraId,subType,uiPosition')
while ( extrasWithCarousel.hasNext() ){
   temp = extrasWithCarousel.next();
   print(temp._id.valueOf() + ',' + temp.subType + ',' + temp.uiPosition)
}
```

In [19]:
extras_df = pd.read_csv('./extras_with_carousel_2017_07_30.csv')
extras_df['uiPosition'].replace('undefined','DEFAULT', inplace=True) # some are undefined but java fallback to default
print extras_df.uiPosition.unique()
extras_df[extras_df['uiPosition'] != 'DEFAULT'].head(20)

['DEFAULT' 'BOTTOM']


,extraId,subType,uiPosition
28,551d1c1ee4b0297fa1beebcd,CAROUSEL,BOTTOM
32,55388b54e4b099bcc1225caa,CAROUSEL,BOTTOM
38,559f6a87e4b0c0b7288dadb6,CAROUSEL,BOTTOM


In [35]:
players_df = players_df.merge(extras_df, left_on='extraIds', right_on='extraId', how='right')
players_df.drop('extraIds', axis=1, inplace=True)
print 'number of players with carousel: ', players_df.shape[0]
players_df.head(2)

number of players with carousel:  9485


,pid,parent_cid,cid,templateId,staticVideos,bidId,playlistId,extraId,subType,uiPosition
0,57223186e4b0af35cc801d63,50d595ec0364e95588c77bd2,50d595ec0364e95588c77bd2,5668ad07e4b0f82be105f939,0.0,572192e8e4b0e7aba837de62,NaN,55388b54e4b099bcc1225caa,CAROUSEL,BOTTOM
1,5730af47e4b059200314525f,5643911267b62318603bce5a,5643911267b62318603bce5a,56c21811e4b009d9eacb4ee9,0.0,NaN,NaN,55388b54e4b099bcc1225caa,CAROUSEL,BOTTOM
